# Infer ***nuclei*** - 1️⃣ 

--------------

## OVERVIEW
We will start by segmenting the different cell regions - the nucleus, cell, and cytoplasm - since they will be necessary for determining which organelle are in which cell. This is integral to our single cell analysis approach.

This notebook goes through the workflow steps to segment the ***nucleus*** from a fluorescent nuclei marker.


## OBJECTIVE: 
### ✅ Infer sub-cellular component #1: ***nuclei***
Segment the ***nuclei*** from a single channel (nuclei marker). This will be necessary to determine the other subcellular compartment - like the ***cytoplasm***. Nuclei will also be used to seed the instance segmentation of the ***cell*** area.

> ***Convention:***  "nuclei" for the segmentation of ALL nuclei in the image.  "nucleus" for the ***single*** nucleus associated to the single cell being analyzed (determined in [02_infer_cellmask_from-composite](./02_infer_cellmask_from-composite.ipynb)).

## IMPORTS

In [ ]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Optional, Union

import numpy as np

from aicssegmentation.core.pre_processing_utils import  image_smoothing_gaussian_slice_by_slice 
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                                                    export_inferred_organelle,
                                                                    list_image_files)

from infer_subc.core.img import *
from infer_subc.constants import (TEST_IMG_N,
                                  NUC_CH ,
                                  LYSO_CH ,
                                  MITO_CH ,
                                  GOLGI_CH ,
                                  PEROX_CH ,
                                  ER_CH ,
                                  LD_CH ,
                                  RESIDUAL_CH ) 

LD_CH = 0
NUC_CH = 1
LYSO_CH = 2
MITO_CH = 3
GOLGI_CH = 4
PEROX_CH = 5
ER_CH = 6
PM_CH = 7
RESIDUAL_CH = 8
TEST_IMG_N = 0

%load_ext autoreload
%autoreload 2

In [ ]:
viewer = napari.Viewer()

## Get and load Image for processing

In [ ]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw"
im_type = ".tiff"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "out"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

print(test_img_name)

In [ ]:
img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

----------
## infer ***nuclei***
 

### summary of steps

➡️ INPUT (extract)
- select single channel containing the nuclei marker (channel number = user input)

PRE-PROCESSING
- rescale image intensities: 
    - min=0, max=1
- smooth image:
    - median filter (media size = user input)
    - gaussian filter (sigma = user input)

CORE PROCESSING
  - calculate threshold value:
      - log transform image
      - calculate Li's minimum cross entropy threshold value
      - inverse log transform threshold value
  - Apply threshold to image:
      - multiple threshold value by threshold factor (user input)
      - compare threshold min and max values (user input) to adjusted threshold value 
      - apply threshold to image

POST-PROCESSING
  - fill holes (hole size = user input)
  - remove small objects (object size = user input)


OUTPUT ➡️ 
  - label unique nuclei objects based on connectivity
  - save labeled ***nuclei*** (nuclei, NU) as unsigned integer 16-bit tif files

## EXTRACT prototype

In [ ]:
###################
# INPUT
###################
raw_nuclei = select_channel_from_raw(img_data, NUC_CH)

In [ ]:
viewer.add_image(img_data)

## PRE-PROCESSING prototype


In [ ]:
###################
# PRE_PROCESSING
###################           

med_filter_size = 4
gaussian_smoothing_sigma = 1.34

nuclei =  scale_and_smooth(raw_nuclei,
                           median_size = med_filter_size, 
                           gauss_sigma = gaussian_smoothing_sigma)

## CORE PROCESSING prototype


> #### ASIDE: Thresholding
> [Thresholding](https://en.wikipedia.org/wiki/Thresholding_%28image_processing%29) is used to create binary images. A threshold value determines the intensity value separating foreground pixels from background pixels. Foregound pixels are pixels brighter than the threshold value, background pixels are darker. In many cases, images can be adequately segmented by thresholding followed by labelling of *connected components*, which is a fancy way of saying "groups of pixels that touch each other".
> 
> Different thresholding algorithms produce different results. [Otsu's method](https://en.wikipedia.org/wiki/Otsu%27s_method) and [Li's minimum cross entropy threshold](https://scikit-image.org/docs/dev/auto_examples/developers/plot_threshold_li.html) are two common algorithms. Below, we use Li. You can use `skimage.filters.threshold_<TAB>` to find different thresholding methods.


In [ ]:
###################
# CORE_PROCESSING
###################
# log transform the image, calculate the threshold value using Li minimum cross entropy method, inverse log transform the value
# apply the threshold value taking into account the user determined min, max, and adjustment values
threshold_factor = 10
thresh_min = 0.11
thresh_max = 1

li_thresholded = apply_log_li_threshold(nuclei, 
                                        thresh_factor=threshold_factor, 
                                        thresh_min=thresh_min, 
                                        thresh_max=thresh_max)

## POST PROCESSING prototype

> NOTE: the size parameters are by convention defined as one dimensional "width", so the inputs to the functions are _squared_ i.e. raised to the power of 2: `**2` for 2D analysis. For volumetric (3D) analysis this would be _cubed_: `**3`.

In [ ]:
###################
# POST_PROCESSING
###################
# fill small holes then exclude small objects
hole_min_width = 0
hole_max_width = 25  

small_object_width = 15

removed_holes = hole_filling_linear_size(li_thresholded, 
                                         hole_min=hole_min_width, 
                                         hole_max=hole_max_width)

cleaned_img = size_filter_linear_size(removed_holes,
                                      min_size=small_object_width,
                                      method="3D")

# combine the above functions into one for downstream use in plugin
cleaned_img2 = fill_and_filter_linear_size(li_thresholded, 
                                           hole_min=hole_min_width, 
                                           hole_max=hole_max_width, 
                                           min_size= small_object_width,
                                           method="3D")

In [ ]:
# make sure it works..
np.all(cleaned_img==cleaned_img2)

## LABELING prototype

In [ ]:
###################
# LABELING
###################
# create instance segmentation based on connectivity
nuclei_labels = label_uint16(cleaned_img2)

In [ ]:
nuclei_labels.dtype

## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer.add_labels(
    nuclei_labels,
    opacity=0.3)

In [ ]:
viewer.add_image(
    nuclei)

viewer.add_image(
    li_thresholded,
    opacity=0.3,)

viewer.add_labels(
    nuclei_labels,
    opacity=0.3)

## SAVE inferred nuclei to .tif file


In [ ]:
out_file_n = export_inferred_organelle(nuclei_labels, "nuclei", meta_dict, out_data_path)

: 

----------
## DEFINE `infer_nuclei_fromlabel` function

Based on the _prototyping_ above define the function to infer nuclei. 

> NOTE: these functions mainly serve for downstream prototyping in the notebooks. Each step above has an independent function that is implemented in the plugin for easy of use.

In [ ]:
##########################
#  _infer_nuclei
##########################
def _infer_nuclei_fromlabel(in_img: np.ndarray, 
                            nuc_ch: Union[int,None],
                            median_sz: int, 
                            gauss_sig: float,
                            thresh_factor: float,
                            thresh_min: float,
                            thresh_max: float,
                            min_hole_w: int,
                            max_hole_w: int,
                            small_obj_w: int,
                            fill_filter_method: str
                            ) -> np.ndarray:
    """
    Procedure to infer nuclei from linear unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    median_sz: int
        width of median filter for signal
    gauss_sig: float
        sigma for gaussian smoothing of  signal
    thresh_factor: float
        adjustment factor for log Li threholding
    thresh_min: float
        abs min threhold for log Li threholding
    thresh_max: float
        abs max threhold for log Li threholding
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimu object size cutoff for nuclei post-processing

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """
    ###################
    # EXTRACT
    ###################                
    nuclei = select_channel_from_raw(in_img, nuc_ch)

    ###################
    # PRE_PROCESSING
    ###################                
    nuclei =  scale_and_smooth(nuclei,
                        median_size = median_sz, 
                        gauss_sigma = gauss_sig)

    ###################
    # CORE_PROCESSING
    ###################
    nuclei_object = apply_log_li_threshold(nuclei, 
                                           thresh_factor=thresh_factor, 
                                           thresh_min=thresh_min, 
                                           thresh_max=thresh_max)

    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = fill_and_filter_linear_size(nuclei_object, 
                                                hole_min=min_hole_w, 
                                                hole_max=max_hole_w, 
                                                min_size=small_obj_w,
                                                method=fill_filter_method)

    nuclei_labels = label_uint16(nuclei_object)

    return nuclei_labels

## DEFINE `_fixed_infer_nuclei` function

Based on the _prototyping_ above define the function to infer cellmask. with a *fixed* set of parameters for each step in the procedure.  That is they are all "hard coded"

In [ ]:
##########################
#  fixed_infer_nuclei
##########################
def _fixed_infer_nuclei_fromlabel(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer cellmask from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
 
    Returns
    -------------
    nuclei_object
        inferred nuclei
    
    """
    nuc_ch = NUC_CH
    median_sz = 4   
    gauss_sig = 1.34
    threshold_factor = 0.9
    thresh_min = 0.1
    thresh_max = 1.0
    min_hole_w = 0
    max_hole_w = 25
    small_obj_w = 15
    fill_filter_method = "3D"

    return _infer_nuclei_fromlabel( in_img,
                                    nuc_ch,
                                    median_sz,
                                    gauss_sig,
                                    threshold_factor,
                                    thresh_min,
                                    thresh_max,
                                    min_hole_w,
                                    max_hole_w,
                                    small_obj_w,
                                    fill_filter_method)


## TEST `_infer_nuclei_fromlabel`  function defined above

In [ ]:

_NU_object =  _fixed_infer_nuclei_fromlabel(img_data) 

_NU_object.dtype

In [ ]:
np.all(nuclei_labels == _NU_object)

In [ ]:
viewer.add_image(_NU_object)

---------------------
# TEST `infer_nuclei_fromlabel` exported functions

The prototype `_infer_nuclei_fromlabel` was copied to the [`.organelles.nuclei`](../infer_subc/organelles/nuclei.py) sub-module.

In [ ]:
from infer_subc.organelles.nuclei import infer_nuclei_fromlabel, fixed_infer_nuclei_fromlabel
nuclei_object =  fixed_infer_nuclei_fromlabel(img_data) 

nuclei_object.dtype

In [ ]:
np.all(_NU_object == nuclei_object)

## Visualize  2


In [ ]:
viewer.add_image(
    nuclei_object,
    opacity=0.3)

viewer.add_image(
    label(nuclei_object),
    opacity=0.3)

In [ ]:
from napari.utils.notebook_display import nbscreenshot

nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()

-------------
## SUMMARY

The above explains the general procedure for segmenting all the nuclei in an image based on a fluorescent nuclei marker (single channel). 

### NEXT: INFER CELLMASK

proceed to [02_infer_cellmask.ipynb](./02_infer_cellmask.ipynb)
